In [1]:
import tensorflow as tf
import numpy as np

tf.enable_eager_execution()

In [2]:
train_images = np.genfromtxt('input/train.csv', delimiter=',', skip_header=1)
test_images = np.genfromtxt('input/test.csv', delimiter=',', skip_header=1)

In [3]:
train_labels = train_images[:,0] # label from first column

In [4]:
train_images = train_images[:,1:]  # remove first column

In [5]:
(train_images.shape, train_labels.shape, test_images.shape)

((42000, 784), (42000,), (28000, 784))

In [6]:
TRAINING_SIZE = len(train_images)
TEST_SIZE = len(test_images)

train_images = np.asarray(train_images, dtype=np.float32) / 255

# Convert the train images and add channels
train_images = train_images.reshape((-1, 28, 28, 1)) #-1 will auto calc to 42000

test_images = np.asarray(test_images, dtype=np.float32) / 255
# Convert the train images and add channels
test_images = test_images.reshape((-1, 28, 28, 1))

In [7]:
(train_images.shape, train_labels.shape, test_images.shape)

((42000, 28, 28, 1), (42000,), (28000, 28, 28, 1))

In [8]:
# How many digits we are predicting from (0-9)
LABEL_DIMENSIONS = 10

train_labels  = tf.keras.utils.to_categorical(train_labels, LABEL_DIMENSIONS)

# Cast the labels to floats, needed later
train_labels = train_labels.astype(np.float32)

In [9]:
model = tf.keras.Sequential()

model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation=tf.nn.relu, input_shape=(28, 28, 1)))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2))
model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation=tf.nn.relu))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2))
model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation=tf.nn.relu))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(64, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(10, activation=tf.nn.softmax))

In [11]:
optimizer = tf.train.RMSPropOptimizer(learning_rate=0.001)
# optimizer = tf.train.AdamOptimizer()  # performs worse than RMSProp

model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                36928     
__________

In [12]:
BATCH_SIZE=128

# Because tf.data may work with potentially **large** collections of data
# we do not shuffle the entire dataset by default
# Instead, we maintain a buffer of SHUFFLE_SIZE elements
# and sample from there.
SHUFFLE_SIZE = 10000 

# Create the dataset
dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
dataset = dataset.shuffle(SHUFFLE_SIZE)
dataset = dataset.batch(BATCH_SIZE)

In [ ]:
EPOCHS = 30

for epoch in range(EPOCHS):
  for (batch, (images, labels)) in enumerate(dataset):
    train_loss, train_accuracy = model.train_on_batch(images, labels)
    
    #if batch % 10 == 0: print(batch, train_accuracy)
  
  # Here you can gather any metrics or adjust your training parameters
  print('Epoch #%d\t Loss: %.6f\tAccuracy: %.6f' % (epoch + 1, train_loss, train_accuracy))

Epoch #1	 Loss: 0.025480	Accuracy: 1.000000
Epoch #2	 Loss: 0.013370	Accuracy: 1.000000
Epoch #3	 Loss: 0.223661	Accuracy: 0.875000
Epoch #4	 Loss: 0.047256	Accuracy: 1.000000
Epoch #5	 Loss: 0.000167	Accuracy: 1.000000
Epoch #6	 Loss: 0.000117	Accuracy: 1.000000
Epoch #7	 Loss: 0.000231	Accuracy: 1.000000
Epoch #8	 Loss: 0.000519	Accuracy: 1.000000
Epoch #9	 Loss: 0.000053	Accuracy: 1.000000
Epoch #10	 Loss: 0.001298	Accuracy: 1.000000
Epoch #11	 Loss: 0.000001	Accuracy: 1.000000
Epoch #12	 Loss: 0.000001	Accuracy: 1.000000
Epoch #13	 Loss: 0.000887	Accuracy: 1.000000
Epoch #14	 Loss: 0.001282	Accuracy: 1.000000
Epoch #15	 Loss: 0.000130	Accuracy: 1.000000
Epoch #16	 Loss: 0.000000	Accuracy: 1.000000
Epoch #17	 Loss: 0.000000	Accuracy: 1.000000
Epoch #18	 Loss: 0.000806	Accuracy: 1.000000


In [ ]:
tf.keras.models.save_model(
    model,
    "model/model",
    include_optimizer=False
)

In [ ]:
md = tf.keras.models.load_model("model/model",compile=False)

In [ ]:
prediction = md.predict(test_images)

In [ ]:
prediction2 = model.predict(test_images)

In [ ]:
(prediction2==prediction).sum()

In [ ]:
label = tf.argmax(prediction,axis=1)
image_id = np.array(np.arange(1,TEST_SIZE+1))
result = np.column_stack((image_id, label))

In [ ]:
np.savetxt("output/cnn_predictions3.csv", result, fmt=('%d,%d'), delimiter=",", \
           header="ImageId,Label",comments='')